In [1]:
#Data preprocessing
import numpy as np
import pandas as pd
import re
from imblearn.over_sampling import SMOTE 

#Data Transformation
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso,Ridge
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier,VotingClassifier
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,RandomForestRegressor
from sklearn.model_selection import train_test_split,StratifiedKFold,KFold
from sklearn.preprocessing import LabelEncoder,StandardScaler,RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,mean_squared_error

#Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Handling warnings
import warnings
warnings.filterwarnings("ignore")

import os
import pickle

C:\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
req_data = input("Enter desired dataset : ")
if req_data not in os.listdir('Samples/'):
    print("No such dataset")
else:
    if re.findall("\.[a-z]+",req_data)[0][1:]=="csv":
        data = pd.read_csv(f'Samples/{req_data}')
    elif re.findall("\.[a-z]+",req_data)[0][1:]=="xlsx":
        data = pd.read_excel(f'Samples/{req_data}')

Enter desired dataset : Iris.csv


In [3]:
data.head(3)

Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa

In [4]:
def BMC(data):
    le = LabelEncoder()
    pipeline = Pipeline([("robust scaler",RobustScaler()),("std_scaler",StandardScaler())])
    X,y = data.iloc[:,:-1],data.iloc[:,-1]
    for col in X.select_dtypes("O").columns:
        X[col] = le.fit_transform(X[col])
    for col in X.columns:
        X[col]=pipeline.fit_transform(np.array(X[col]).reshape(-1,1))
    if y.dtype in [object]:
        estimators = {"KNearestNeighbor":KNeighborsClassifier(),
                     "Support Vector Regressor":SVC(),"DecisionTreeClassifier":DecisionTreeClassifier(),
                     "AdaBoostClassifier":AdaBoostClassifier(),
                     "GradientBoostingClassifier":GradientBoostingClassifier(),
                     "LogisticRegression":LogisticRegression()}
        str_kfold = StratifiedKFold(n_splits=10)
        model_train_scores,model_test_scores = [],[]
        for estimator in estimators.values():
            train_scores,test_scores = [],[]
            for train_index,test_index in str_kfold.split(X,y):
                X_train,X_test = X.iloc[train_index,:],X.iloc[test_index,:]
                y_train,y_test = y.iloc[train_index],y.iloc[test_index]
                estimator.fit(X_train,y_train)
                train_scores.append(estimator.score(X_train,y_train))
                test_scores.append(estimator.score(X_test,y_test))
            model_train_scores.append(np.array(train_scores).mean())
            model_test_scores.append(np.array(test_scores).mean())
        df = pd.DataFrame({"Model":estimators.values(),
                 "Train Score":model_train_scores,
                 "Test Score":model_test_scores})
        df["Model"] = df["Model"].apply(lambda x:str(x)[:-2])
        df = df.nlargest(8,columns=["Train Score","Test Score"])
    elif y.dtype in [float,int]:
        estimators = {"KNearestNeighbor":KNeighborsRegressor(),
                     "SupportVectorRegressor":SVR(),"DecisionTreeRegressor":DecisionTreeRegressor(),
                     "AdaBoostRegressor":AdaBoostRegressor(),
                     "GradientBoostingRegressor":GradientBoostingRegressor(),
                     "LinearRegression":LinearRegression(),
                     "LassoRegression":Lasso(),"RidgeRegression":Ridge()}
        kfold = KFold(n_splits=10)
        model_train_errors,model_test_errors = [],[]
        for estimator in estimators.values():
            train_errors,test_errors = [],[]
            for train_index,test_index in kfold.split(X,y):
                X_train,X_test = X.iloc[train_index,:],X.iloc[test_index,:]
                y_train,y_test = y.iloc[train_index],y.iloc[test_index]
                estimator.fit(X_train,y_train)
                train_pred = estimator.predict(X_train)
                test_pred = estimator.predict(X_test)
                train_errors.append(mean_squared_error(y_train,train_pred))
                test_errors.append(mean_squared_error(y_test,test_pred))
            model_train_errors.append(np.array(train_errors).mean())
            model_test_errors.append(np.array(test_errors).mean())
        df = pd.DataFrame({"Model":estimators.values(),
                     "Train Errors":np.sqrt(np.array(model_train_errors)),
                     "Test Errors":np.sqrt(np.array(model_test_errors))})
        df["Model"] = df["Model"].apply(lambda x:str(x)[:-2])
        df = df.nlargest(8,columns=["Train Errors","Test Errors"])
    return df